In [1]:
from typing import List
from export_utils import *
import sys
sys.path.append('../')
from models import CNN
from executorch.sdk.bundled_program.core import BundledProgram # type: ignore
from executorch.sdk.bundled_program.config import ( # type: ignore
    MethodInputType,
    MethodTestCase,
    MethodTestSuite,
)

from executorch.sdk.bundled_program.serialize import ( # type: ignore
    serialize_from_bundled_program_to_flatbuffer,
)
import warnings
warnings.filterwarnings("ignore")

In [3]:
model = CNN("mnist", num_classes=10)
model.load("mnist")
print("Exporting executorch program...")
executorch_program : ExecutorchProgramManager = export_to_exec_prog(model=model, example_inputs=model.example_args)

MODEL LOADED!
Exporting executorch program...


In [5]:
print("Creating bundled test cases...")
method_names = [
    method.name for method in executorch_program.executorch_program.execution_plan
]
program_inputs = {
    m_name: [model.example_args] for m_name in method_names
}
method_test_suites: List[MethodTestSuite] = []
for m_name in method_names:
    method_inputs = program_inputs[m_name]
    
    # To create a bundled program, we first create every test cases from input. We leverage eager model
    # to generate expected output for each test input, and use MethodTestCase to hold the information of
    # each test case. We gather all MethodTestCase for same method into one MethodTestSuite, and generate
    # bundled program by all MethodTestSuites.
    method_test_cases: List[MethodTestCase] = []
    for method_input in method_inputs:
        method_test_cases.append(
            MethodTestCase(
                inputs=method_input,
                expected_outputs=model(*method_input),
            )
        )
        
    method_test_suites.append(
        MethodTestSuite(
            method_name=m_name,
            test_cases=method_test_cases,
        )
    )


Creating bundled test cases...


In [6]:
print("Creating bundled program...")   
bundled_program = BundledProgram(executorch_program, method_test_suites)
bundled_program_buffer = serialize_from_bundled_program_to_flatbuffer(
    bundled_program
)
# with open(f"../cpp/cnn_ops/models/{model.name}/bundled.bpte", "wb") as file:
#     file.write(bundled_program_buffer)

Creating bundled program...
